## All files Null columns dropping


In [11]:
import pandas as pd
import numpy as np

def drop_fully_missing_columns(input_file, output_file):
    """
    Import dataset and drop columns with 100% missing values
    """
    try:
        # Read the CSV file
        print(f"Reading data from {input_file}...")
        df = pd.read_csv(input_file)
        
        print(f"Original data shape: {df.shape}")
        print(f"Original columns: {len(df.columns)}")
        
        # Calculate missing values percentage for each column
        missing_percent = (df.isnull().sum() / len(df)) * 100
        fully_missing_cols = missing_percent[missing_percent == 100].index.tolist()
        
        print(f"\nColumns with 100% missing values: {len(fully_missing_cols)}")
        if fully_missing_cols:
            print("Columns to be dropped:")
            for col in fully_missing_cols:
                print(f"  - {col}")
        
        # Drop columns with 100% missing values
        df_cleaned = df.drop(columns=fully_missing_cols)
        
        print(f"\nCleaned data shape: {df_cleaned.shape}")
        print(f"Remaining columns: {len(df_cleaned.columns)}")
        
        # Save the cleaned dataset
        df_cleaned.to_csv(output_file, index=False)
        print(f"\nCleaned dataset saved to: {output_file}")
        
        # Display summary of missing values in remaining columns
        remaining_missing = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
        remaining_missing = remaining_missing[remaining_missing > 0]
        
        if len(remaining_missing) > 0:
            print(f"\nRemaining columns with some missing values: {len(remaining_missing)}")
            for col, percent in remaining_missing.items():
                print(f"  - {col}: {percent:.2f}% missing")
        else:
            print("\nNo missing values in remaining columns!")
        
        return df_cleaned
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

input_file = "csv/mutations.csv"  
output_file = "csv/mutations.csv"
cleaned_df = drop_fully_missing_columns(input_file, output_file)

Reading data from csv/mutations.csv...


C:\Users\ibrahim\AppData\Local\Temp\ipykernel_8784\311224176.py:11: DtypeWarning: Columns (50,146,196) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Original data shape: (13180, 345)
Original columns: 345

Columns with 100% missing values: 20
Columns to be dropped:
  - Match_Norm_Seq_Allele1
  - Match_Norm_Seq_Allele2
  - Tumor_Validation_Allele1
  - Tumor_Validation_Allele2
  - Match_Norm_Validation_Allele1
  - Match_Norm_Validation_Allele2
  - Score
  - BAM_file
  - i_1000gp3_CS
  - i_1000gp3_END
  - i_1000gp3_MC
  - i_1000gp3_MEND
  - i_1000gp3_MLEN
  - i_1000gp3_MSTART
  - i_1000gp3_SVLEN
  - i_1000gp3_SVTYPE
  - i_1000gp3_TSD
  - i_ESP_GWAS_PUBMED
  - i_HGNC_Primary IDs
  - i_HGNC_Secondary IDs

Cleaned data shape: (13180, 325)
Remaining columns: 325

Cleaned dataset saved to: csv/mutations.csv

Remaining columns with some missing values: 279
  - dbSNP_RS: 86.57% missing
  - dbSNP_Val_Status: 96.92% missing
  - Annotation_Transcript: 0.14% missing
  - Transcript_Strand: 0.14% missing
  - Transcript_Exon: 0.14% missing
  - Transcript_Position: 0.14% missing
  - cDNA_Change: 2.85% missing
  - Codon_Change: 1.63% missing
  - Prot

## pathology_detail.csv column cleaning

In [1]:
import pandas as pd
import numpy as np

def clean_pathology_data(file_path):
    """
    Load pathology data and drop columns that are completely filled with '-- values
    """
    
    # Read the CSV file
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    
    print(f"Original data shape: {df.shape}")
    print(f"Original columns: {len(df.columns)}")
    
    # Identify columns that are completely filled with '--
    columns_to_drop = []
    
    for column in df.columns:
        # Check if all values in the column are '--
        if (df[column] == "'--").all():
            columns_to_drop.append(column)
            print(f"Column to drop: {column} - completely filled with '--")
    
    # Drop the identified columns
    if columns_to_drop:
        df_cleaned = df.drop(columns=columns_to_drop)
        print(f"\nDropped {len(columns_to_drop)} columns that were completely null ('--)")
    else:
        df_cleaned = df
        print("No completely null columns found to drop")
    
    print(f"Cleaned data shape: {df_cleaned.shape}")
    print(f"Remaining columns: {len(df_cleaned.columns)}")
    
    return df_cleaned, columns_to_drop

def analyze_null_patterns(df):
    """
    Analyze the pattern of null values in the dataset
    """
    print("\n" + "="*50)
    print("NULL VALUE ANALYSIS")
    print("="*50)
    
    # Count '-- values in each column
    null_counts = {}
    total_rows = len(df)
    
    for column in df.columns:
        null_count = (df[column] == "'--").sum()
        null_percentage = (null_count / total_rows) * 100
        null_counts[column] = {
            'null_count': null_count,
            'null_percentage': null_percentage
        }
    
    # Sort by null percentage
    sorted_null_counts = dict(sorted(null_counts.items(), 
                                   key=lambda x: x[1]['null_percentage'], 
                                   reverse=True))
    
    print("\nColumns with highest percentage of '-- values:")
    print("-" * 60)
    print(f"{'Column':<50} {'Null Count':<12} {'Null %':<10}")
    print("-" * 60)
    
    for col, stats in list(sorted_null_counts.items())[:20]:  # Top 20
        print(f"{col:<50} {stats['null_count']:<12} {stats['null_percentage']:<10.1f}")
    
    return sorted_null_counts

def create_cleaned_dataset(file_path, output_path=None):
    """
    Complete pipeline to clean the pathology data
    """
    # Step 1: Clean the data
    df_cleaned, dropped_columns = clean_pathology_data(file_path)
    
    # Step 2: Analyze null patterns
    null_analysis = analyze_null_patterns(df_cleaned)
    
    # Step 3: Additional cleaning - replace remaining '-- with actual NaN
    print("\n" + "="*50)
    print("REPLACING REMAINING '-- WITH NaN")
    print("="*50)
    
    # Replace all remaining '-- with NaN for better pandas handling
    df_final = df_cleaned.replace("'--", np.nan)
    
    # Count how many values were replaced
    total_replaced = df_final.isna().sum().sum() - df_cleaned.isna().sum().sum()
    print(f"Replaced {total_replaced} '-- values with NaN")
    
    # Step 4: Save the cleaned data
    if output_path:
        df_final.to_csv(output_path, index=False)
        print(f"\nCleaned data saved to: {output_path}")
    
    # Step 5: Generate summary report
    print("\n" + "="*50)
    print("CLEANING SUMMARY REPORT")
    print("="*50)
    print(f"Original shape: {pd.read_csv(file_path).shape}")
    print(f"Final shape: {df_final.shape}")
    print(f"Columns dropped: {len(dropped_columns)}")
    print(f"Total null values after cleaning: {df_final.isna().sum().sum()}")
    print(f"Data completeness: {(1 - df_final.isna().sum().sum() / (df_final.shape[0] * df_final.shape[1])) * 100:.1f}%")
    
    # Show remaining columns with high null rates (optional further cleaning)
    high_null_cols = {k: v for k, v in null_analysis.items() if v['null_percentage'] > 90}
    if high_null_cols:
        print(f"\nColumns with >90% null values (consider dropping): {len(high_null_cols)}")
        for col in list(high_null_cols.keys())[:10]:  # Show first 10
            print(f"  - {col}")
    
    return df_final, dropped_columns

# Alternative function for more aggressive cleaning
def aggressive_clean(file_path, null_threshold=95, output_path=None):
    """
    More aggressive cleaning that drops columns with high percentage of null values
    """
    df = pd.read_csv(file_path)
    
    # Replace '-- with NaN first
    df = df.replace("'--", np.nan)
    
    print(f"Original shape: {df.shape}")
    
    # Calculate null percentage for each column
    null_percentages = (df.isnull().sum() / len(df)) * 100
    
    # Identify columns to drop based on threshold
    columns_to_drop = null_percentages[null_percentages > null_threshold].index.tolist()
    
    print(f"\nDropping columns with >{null_threshold}% null values:")
    for col in columns_to_drop:
        print(f"  - {col} ({null_percentages[col]:.1f}% null)")
    
    # Drop columns
    df_cleaned = df.drop(columns=columns_to_drop)
    
    print(f"\nAfter aggressive cleaning:")
    print(f"  Shape: {df_cleaned.shape}")
    print(f"  Columns dropped: {len(columns_to_drop)}")
    print(f"  Remaining columns: {len(df_cleaned.columns)}")
    
    if output_path:
        df_cleaned.to_csv(output_path, index=False)
        print(f"Saved to: {output_path}")
    
    return df_cleaned, columns_to_drop

# Usage examples:

if True:
    file_path = "csv/pathology_detail.csv"
    
    print("METHOD 1: Drop only completely null columns")
    print("="*60)
    df_cleaned1, dropped1 = create_cleaned_dataset(
        file_path, 
        output_path="pathology_detail_cleaned.csv"
    )
    print("\n" + "="*60)
    print("METHOD 2: Aggressive cleaning (drop high null columns)")
    print("="*60)
    df_cleaned2, dropped2 = aggressive_clean(
        file_path,
        null_threshold=95,  # Drop columns with >95% null values
        output_path="pathology_detail_aggressive_cleaned.csv"
    )

METHOD 1: Drop only completely null columns
Loading data from csv/pathology_detail.csv...
Original data shape: (175, 86)
Original columns: 86
Column to drop: pathology_details.additional_pathology_findings - completely filled with '--
Column to drop: pathology_details.anaplasia_present - completely filled with '--
Column to drop: pathology_details.anaplasia_present_type - completely filled with '--
Column to drop: pathology_details.bone_marrow_malignant_cells - completely filled with '--
Column to drop: pathology_details.breslow_thickness - completely filled with '--
Column to drop: pathology_details.circumferential_resection_margin - completely filled with '--
Column to drop: pathology_details.columnar_mucosa_present - completely filled with '--
Column to drop: pathology_details.days_to_pathology_detail - completely filled with '--
Column to drop: pathology_details.dysplasia_degree - completely filled with '--
Column to drop: pathology_details.dysplasia_type - completely filled with '

C:\Users\ibrahim\AppData\Local\Temp\ipykernel_10540\678921372.py:127: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("'--", np.nan)


## For Exposure file

In [4]:
import pandas as pd
import numpy as np

def clean_pathology_data(file_path):
    """
    Load pathology data and drop columns that are completely filled with '-- values
    """
    
    # Read the CSV file
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    
    print(f"Original data shape: {df.shape}")
    print(f"Original columns: {len(df.columns)}")
    
    # Identify columns that are completely filled with '--
    columns_to_drop = []
    
    for column in df.columns:
        # Check if all values in the column are '--
        if (df[column] == "-").all():
            columns_to_drop.append(column)
            print(f"Column to drop: {column} - completely filled with '--")
    
    # Drop the identified columns
    if columns_to_drop:
        df_cleaned = df.drop(columns=columns_to_drop)
        print(f"\nDropped {len(columns_to_drop)} columns that were completely null ('--)")
    else:
        df_cleaned = df
        print("No completely null columns found to drop")
    
    print(f"Cleaned data shape: {df_cleaned.shape}")
    print(f"Remaining columns: {len(df_cleaned.columns)}")
    
    return df_cleaned, columns_to_drop

def analyze_null_patterns(df):
    """
    Analyze the pattern of null values in the dataset
    """
    print("\n" + "="*50)
    print("NULL VALUE ANALYSIS")
    print("="*50)
    
    # Count '-- values in each column
    null_counts = {}
    total_rows = len(df)
    
    for column in df.columns:
        null_count = (df[column] == "'--").sum()
        null_percentage = (null_count / total_rows) * 100
        null_counts[column] = {
            'null_count': null_count,
            'null_percentage': null_percentage
        }
    
    # Sort by null percentage
    sorted_null_counts = dict(sorted(null_counts.items(), 
                                   key=lambda x: x[1]['null_percentage'], 
                                   reverse=True))
    
    print("\nColumns with highest percentage of '-- values:")
    print("-" * 60)
    print(f"{'Column':<50} {'Null Count':<12} {'Null %':<10}")
    print("-" * 60)
    
    for col, stats in list(sorted_null_counts.items())[:20]:  # Top 20
        print(f"{col:<50} {stats['null_count']:<12} {stats['null_percentage']:<10.1f}")
    
    return sorted_null_counts

def create_cleaned_dataset(file_path, output_path=None):
    """
    Complete pipeline to clean the pathology data
    """
    # Step 1: Clean the data
    df_cleaned, dropped_columns = clean_pathology_data(file_path)
    
    # Step 2: Analyze null patterns
    null_analysis = analyze_null_patterns(df_cleaned)
    
    # Step 3: Additional cleaning - replace remaining '-- with actual NaN
    print("\n" + "="*50)
    print("REPLACING REMAINING '-- WITH NaN")
    print("="*50)
    
    # Replace all remaining '-- with NaN for better pandas handling
    df_final = df_cleaned.replace("'--", np.nan)
    
    # Count how many values were replaced
    total_replaced = df_final.isna().sum().sum() - df_cleaned.isna().sum().sum()
    print(f"Replaced {total_replaced} '-- values with NaN")
    
    # Step 4: Save the cleaned data
    if output_path:
        df_final.to_csv(output_path, index=False)
        print(f"\nCleaned data saved to: {output_path}")
    
    # Step 5: Generate summary report
    print("\n" + "="*50)
    print("CLEANING SUMMARY REPORT")
    print("="*50)
    print(f"Original shape: {pd.read_csv(file_path).shape}")
    print(f"Final shape: {df_final.shape}")
    print(f"Columns dropped: {len(dropped_columns)}")
    print(f"Total null values after cleaning: {df_final.isna().sum().sum()}")
    print(f"Data completeness: {(1 - df_final.isna().sum().sum() / (df_final.shape[0] * df_final.shape[1])) * 100:.1f}%")
    
    # Show remaining columns with high null rates (optional further cleaning)
    high_null_cols = {k: v for k, v in null_analysis.items() if v['null_percentage'] > 90}
    if high_null_cols:
        print(f"\nColumns with >90% null values (consider dropping): {len(high_null_cols)}")
        for col in list(high_null_cols.keys())[:10]:  # Show first 10
            print(f"  - {col}")
    
    return df_final, dropped_columns

# Alternative function for more aggressive cleaning
def aggressive_clean(file_path, null_threshold=95, output_path=None):
    """
    More aggressive cleaning that drops columns with high percentage of null values
    """
    df = pd.read_csv(file_path)
    
    # Replace '-- with NaN first
    df = df.replace("-", np.nan)
    
    print(f"Original shape: {df.shape}")
    
    # Calculate null percentage for each column
    null_percentages = (df.isnull().sum() / len(df)) * 100
    
    # Identify columns to drop based on threshold
    columns_to_drop = null_percentages[null_percentages > null_threshold].index.tolist()
    
    print(f"\nDropping columns with >{null_threshold}% null values:")
    for col in columns_to_drop:
        print(f"  - {col} ({null_percentages[col]:.1f}% null)")
    
    # Drop columns
    df_cleaned = df.drop(columns=columns_to_drop)
    
    print(f"\nAfter aggressive cleaning:")
    print(f"  Shape: {df_cleaned.shape}")
    print(f"  Columns dropped: {len(columns_to_drop)}")
    print(f"  Remaining columns: {len(df_cleaned.columns)}")
    
    if output_path:
        df_cleaned.to_csv(output_path, index=False)
        print(f"Saved to: {output_path}")
    
    return df_cleaned, columns_to_drop

# Usage examples:

if True:
    file_path = "csv/exposure.csv"
    
    print("METHOD 1: Drop only completely null columns")
    print("="*60)
    df_cleaned1, dropped1 = create_cleaned_dataset(
        file_path, 
        output_path="exposure.csv"
    )

METHOD 1: Drop only completely null columns
Loading data from csv/exposure.csv...
Original data shape: (114, 40)
Original columns: 40
Column to drop: exposures.age_at_last_exposure - completely filled with '--
Column to drop: exposures.age_at_onset - completely filled with '--
Column to drop: exposures.alcohol_frequency - completely filled with '--
Column to drop: exposures.alcohol_intensity - completely filled with '--
Column to drop: exposures.alcohol_type - completely filled with '--
Column to drop: exposures.asbestos_exposure - completely filled with '--
Column to drop: exposures.asbestos_exposure_type - completely filled with '--
Column to drop: exposures.chemical_exposure_type - completely filled with '--
Column to drop: exposures.cigarettes_per_day - completely filled with '--
Column to drop: exposures.coal_dust_exposure - completely filled with '--
Column to drop: exposures.environmental_tobacco_smoke_exposure - completely filled with '--
Column to drop: exposures.exposure_dura

## For merging 3 datasets and cleaning

In [1]:
import pandas as pd
clinical = pd.read_csv("csv/clinical.csv", sep=",")
exposure = pd.read_csv("csv/exposure.csv", sep=",")
follow_up = pd.read_csv("csv/follow_up.csv", sep=",")
key = "cases.case_id"
if key not in clinical.columns:
    raise KeyError(f"{key} not found in clinical.txt")
if key not in exposure.columns:
    raise KeyError(f"{key} not found in exposure.txt")
if key not in follow_up.columns:
    raise KeyError(f"{key} not found in follow_up.txt")

clinical[key] = clinical[key].astype(str)
exposure[key] = exposure[key].astype(str)
follow_up[key] = follow_up[key].astype(str)

# -------------------------
# Merge datasets with FULL OUTER JOIN (no data loss)
# -------------------------
merged = clinical.merge(exposure, on=key, how="outer")
merged = merged.merge(follow_up, on=key, how="outer")
merged.to_csv("merged_clinical_exposure_followup.csv", index=False)
print("Merged dataset created successfully!")
print("Shape:", merged.shape)

Merged dataset created successfully!
Shape: (1779, 102)


In [2]:
import pandas as pd

# ---------------------------------------------------------
# Load your merged dataset
# ---------------------------------------------------------
df = pd.read_csv("merged_clinical_exposure_followup.csv")

# ---------------------------------------------------------
# 1. Sort by `cases.case_id` and `cases.submitter_id_x`
# ---------------------------------------------------------
# Ensure both columns exist
if "cases.case_id" not in df.columns:
    raise KeyError("Column 'cases.case_id' not found!")

if "cases.submitter_id_x" not in df.columns:
    raise KeyError("Column 'cases.submitter_id_x' not found!")

# Sort
df = df.sort_values(by=["cases.case_id", "cases.submitter_id_x"], ascending=[True, True])

# Move the two columns to the front next to each other
cols = df.columns.tolist()
cols.remove("cases.case_id")
cols.remove("cases.submitter_id_x")
df = df[["cases.case_id", "cases.submitter_id_x"] + cols]

# ---------------------------------------------------------
# 2. Concatenate cases.disease_type + cases.index_date
# ---------------------------------------------------------
if "cases.disease_type" in df.columns and "cases.index_date" in df.columns:
    df["cases.disease_index"] = df["cases.disease_type"].fillna("") + " | " + df["cases.index_date"].fillna("")
    df["cases.disease_index"] = df["cases.disease_index"].str.strip(" |")
else:
    print("Warning: 'cases.disease_type' or 'cases.index_date' not found.")

# ---------------------------------------------------------
# 3. Combine demographic.ethnicity + demographic.race
# ---------------------------------------------------------
eth_col = "demographic.ethnicity"
race_col = "demographic.race"

if eth_col in df.columns and race_col in df.columns:
    df["demographic.ethnicity_race"] = (
        df[eth_col].fillna("") + " - " + df[race_col].fillna("")
    ).str.strip(" -")
else:
    print("Warning: Ethnicity or Race column missing!")

# ---------------------------------------------------------
# Save cleaned output
# ---------------------------------------------------------
df.to_csv("merged_clinical_exposure_followup_CLEAN.csv", index=False)

print("✔ Finished! Output saved as merged_clinical_exposure_followup_CLEAN.csv")


✔ Finished! Output saved as merged_clinical_exposure_followup_CLEAN.csv


## Combining the transcriptomics.csv and methylation.csv

In [1]:
import pandas as pd

# ----------------------------------------------
# 1. Load datasets
# ----------------------------------------------
df_trans = pd.read_csv("csv/transcriptomics.csv")
df_meth  = pd.read_csv("csv/methylation.csv")

df_meth = df_meth.rename(columns={'V1': 'gene_id'})

# ----------------------------------------------
# 2. Normalize TCGA IDs
# ----------------------------------------------
def normalize_id(col):
    if isinstance(col, str) and col.startswith("TCGA"):
        parts = col.split("-")
        if len(parts) >= 4:
            return f"{parts[0]}-{parts[1]}-{parts[2]}-{parts[3][:2]}"
    return col

df_trans.columns = [normalize_id(c) for c in df_trans.columns]
df_meth.columns  = [normalize_id(c) for c in df_meth.columns]

# ----------------------------------------------
# 3. Find common samples
# ----------------------------------------------
trans_samples = set(df_trans.columns) - {"gene_id"}
meth_samples  = set(df_meth.columns) - {"gene_id"}

common_samples = sorted(trans_samples.intersection(meth_samples))
print("Common matched samples:", len(common_samples))

# ----------------------------------------------
# 4. Find common GENES (IMPORTANT FIX)
# ----------------------------------------------
common_genes = sorted(set(df_trans["gene_id"]).intersection(df_meth["gene_id"]))
print("Common genes:", len(common_genes))

# Filter both to common genes only
df_trans_f = df_trans[df_trans["gene_id"].isin(common_genes)]
df_meth_f  = df_meth[df_meth["gene_id"].isin(common_genes)]

# Set index to gene_id
trans = df_trans_f.set_index("gene_id")[common_samples]
meth  = df_meth_f.set_index("gene_id")[common_samples]

# Now they match 1-to-1 without KeyError
print("Transcriptomics aligned shape:", trans.shape)
print("Methylation aligned shape:", meth.shape)

# ----------------------------------------------
# 5. Combined score
# ----------------------------------------------
combined = (1 - meth) * trans

combined.to_csv("combined_epigenetic_score_matrix.csv")

print("Final combined shape:", combined.shape)
combined.head()


Common matched samples: 88
Common genes: 19223
Transcriptomics aligned shape: (19224, 88)
Methylation aligned shape: (19223, 88)
Final combined shape: (19224, 88)


,TCGA-BA-4074-01,TCGA-BA-4077-01,TCGA-BA-5153-01,TCGA-BA-5556-01,TCGA-BA-5557-01,TCGA-BA-6872-01,TCGA-BA-6873-01,TCGA-BA-7269-01,TCGA-BB-4224-01,TCGA-BB-4225-01,...,TCGA-CX-7082-01,TCGA-CX-7086-01,TCGA-D6-6516-01,TCGA-D6-6823-01,TCGA-DQ-5624-01,TCGA-DQ-5625-01,TCGA-DQ-5631-01,TCGA-DQ-7588-01,TCGA-H7-7774-01,TCGA-HD-7831-01
gene_id,,,,,,,,,,,,,,,,,,,,,
1-Dec,248.493765,4.758282,0.365159,6.657963,0.805727,4.849233,5.022764,1.704351,12.839721,0.476125,...,8.573603,10.687611,5.656634,23.265784,2.689164,1.858466,16.826791,59.067935,0.531164,21.919849
1-Mar,136.125609,419.006642,75.269811,355.740175,369.288649,26.215438,38.651937,8.460583,217.069578,276.526602,...,255.177937,95.467913,181.188416,64.200249,157.579292,332.350287,292.322759,29.911578,132.367060,82.262493
1-Sep,149.718523,498.166243,1101.357375,1373.367825,337.641897,186.749782,221.266162,232.157075,158.134833,1375.880360,...,504.807808,272.066124,531.682620,259.547108,408.340166,928.326002,166.187729,97.244087,391.518146,162.325708
10-Mar,114.215866,6.736469,3.366613,3.924946,1.449906,5.851599,29.419235,5.599466,46.273094,0.000000,...,3.656245,0.582764,6.706357,3.541566,0.000000,0.834903,8.906240,20.022391,0.000000,4.607995
10-Sep,14402.328014,11241.679840,9136.669608,9255.783136,8942.622941,3998.016649,4074.463859,3740.011866,10427.833554,5448.996823,...,10138.991480,11919.509939,7295.099708,10891.616299,11828.240871,11197.224660,20315.463867,7820.665859,7763.630358,5902.336920


In [2]:
import pandas as pd
import numpy as np

# Load combined matrix
df = pd.read_csv("combined_epigenetic_score_matrix.csv")

# Remove Excel-destroyed "date" gene names
bad_rows = df['gene_id'].str.contains(r'^\d{1,2}[-/][A-Za-z]{3}$', regex=True, na=False)

df_clean = df[~bad_rows]

df_clean.to_csv("combined_epigenetic_score_matrix.csv", index=False)

print("Removed rows:", bad_rows.sum())
print("Final shape:", df_clean.shape)


Removed rows: 25
Final shape: (19199, 89)


## Combine Pathology and Cohorts data

In [7]:
import pandas as pd
import json

# -----------------------------
# CONFIG
# -----------------------------
MERGED_CLINICAL_PATH = "merged_TCGA_Cohorts.csv"
PATHOLOGY_PATH = "csv/pathology_detail.csv"

OUT_FULL_CLEAN = "merged_clinical_pathology_cleaned_full.csv"
OUT_CURATED = "merged_clinical_pathology_curated_subset.csv"
OUT_DROP_REPORT = "merged_clinical_pathology_drop_report.csv"

# Thresholds for dropping
MISSINGNESS_THRESHOLD = 0.95   # drop if >95% values are NaN
LOW_VARIANCE_UNIQUE_MAX = 1    # drop if <= 1 unique non-null value

# -----------------------------
# 1. LOAD & MERGE
# -----------------------------
print("Loading data...")
merged = pd.read_csv(MERGED_CLINICAL_PATH)
path = pd.read_csv(PATHOLOGY_PATH)

print(f"merged_TCGA_Cohorts shape: {merged.shape}")
print(f"pathology_detail shape:   {path.shape}")

# Merge on cases.case_id (LEFT: keep all clinical cases)
df = merged.merge(path, on="cases.case_id", how="left")
print(f"\nAfter merge shape: {df.shape}")

# -----------------------------
# 2. UNIFY IDENTIFIERS
# -----------------------------
drop_reasons = {}  # column_name -> reason string

# If both submitter_id versions exist and are identical, unify them
if "cases.submitter_id_x" in df.columns and "cases.submitter_id" in df.columns:
    same = (df["cases.submitter_id_x"] == df["cases.submitter_id"]) | \
           (df["cases.submitter_id"].isna())
    # Only assert if they're mostly identical; otherwise just prefer the clinical one
    if same.mean() > 0.99:
        # Use the clinical one (x) as the main submitter_id
        df["cases.submitter_id"] = df["cases.submitter_id_x"]
    # Drop the helper column
    drop_reasons["cases.submitter_id_x"] = "manual_redundant: duplicate of cases.submitter_id"
    df.drop(columns=["cases.submitter_id_x"], inplace=True, errors="ignore")

# -----------------------------
# 3. CREATE COMBINED / HELPER COLUMNS
# -----------------------------
# Combined disease_type + index_date, if both exist
if "cases.disease_type" in df.columns and "cases.index_date" in df.columns:
    if "cases.disease_index" not in df.columns:
        df["cases.disease_index"] = (
            df["cases.disease_type"].astype(str) + " | " +
            df["cases.index_date"].astype(str)
        )

# Combined race + ethnicity
if "demographic.ethnicity" in df.columns and "demographic.race" in df.columns:
    if "demographic.ethnicity_race" not in df.columns:
        df["demographic.ethnicity_race"] = (
            df["demographic.ethnicity"].astype(str) + " | " +
            df["demographic.race"].astype(str)
        )

# -----------------------------
# 4. AUTOMATIC CLEANING (Option A)
#    - All-null
#    - High missingness
#    - Low variance
# -----------------------------
print("\nComputing column statistics...")

n_rows, n_cols = df.shape
null_frac = df.isna().mean()
nunique = df.nunique(dropna=True)

cols_all_null = null_frac[null_frac == 1.0].index.tolist()
cols_high_missing = null_frac[(null_frac > MISSINGNESS_THRESHOLD) &
                              (null_frac < 1.0)].index.tolist()
cols_low_variance = nunique[nunique <= LOW_VARIANCE_UNIQUE_MAX].index.tolist()

# We don't want to drop key columns or obviously important ones, even if low variance
force_keep = {
    "cases.case_id",
    "cases.submitter_id",
    "cases.primary_site",
    "cases.disease_type",
    "cases.index_date",
    "cases.disease_index",
    "demographic.age_at_index",
    "demographic.gender",
    "demographic.race",
    "demographic.ethnicity",
    "demographic.ethnicity_race",
    "demographic.vital_status",
    "demographic.days_to_death",
    "diagnoses.age_at_diagnosis",
    "diagnoses.primary_diagnosis",
    "diagnoses.morphology",
    "diagnoses.tumor_stage",
    "diagnoses.tumor_grade",
    "diagnoses.residual_disease",
    "diagnoses.days_to_last_follow_up",
    "diagnoses.progression_or_recurrence",
    "exposures.pack_years_smoked",
    "exposures.cigarettes_per_day",
    "exposures.alcohol_history",
    "exposures.tobacco_smoking_status",
    "pathology_details.margin_status",
    "pathology_details.lymph_nodes_positive",
    "pathology_details.lymph_nodes_tested",
    "pathology_details.lymphatic_invasion_present",
    "pathology_details.perineural_invasion_present",
    "pathology_details.vascular_invasion_present",
}

# Ensure we only drop columns that are not forced to keep
def filter_drop_list(cols, label):
    filtered = []
    for c in cols:
        if c in force_keep:
            continue
        filtered.append(c)
        # If a reason is not already assigned (manual), assign this one
        if c not in drop_reasons:
            drop_reasons[c] = f"{label}"
    return filtered

cols_all_null = filter_drop_list(cols_all_null, "all_null")
cols_high_missing = filter_drop_list(cols_high_missing, "high_missing")
cols_low_variance = filter_drop_list(cols_low_variance, "low_variance")

# Combine all drop lists without duplication
cols_to_drop = list(set(cols_all_null + cols_high_missing + cols_low_variance))

print(f"\nColumns to drop (total {len(cols_to_drop)}):")
print(sorted(cols_to_drop))

# Actually drop them
df_clean = df.drop(columns=cols_to_drop, errors="ignore")
print(f"\nShape before clean: {n_rows} rows x {n_cols} cols")
print(f"Shape after  clean: {df_clean.shape[0]} rows x {df_clean.shape[1]} cols")

# -----------------------------
# 5. BUILD DROP REPORT
# -----------------------------
# Build a dataframe with stats + reason for each dropped column
rows = []
for col in cols_to_drop:
    rows.append({
        "column": col,
        "reason": drop_reasons.get(col, "dropped"),
        "null_fraction": float(null_frac.get(col, float("nan"))),
        "n_unique_non_null": int(nunique.get(col, 0))
    })

drop_report_df = pd.DataFrame(rows).sort_values("column")
drop_report_df.to_csv(OUT_DROP_REPORT, index=False)
print(f"\nDrop report saved to: {OUT_DROP_REPORT}")

# -----------------------------
# 6. BUILD CURATED, SMALLER DATASET
#    (useful + related clinical & pathology columns)
# -----------------------------
curated_cols = [
    # IDs
    "cases.case_id",
    "cases.submitter_id",
    "cases.primary_site",
    "cases.disease_type",
    "cases.index_date",
    "cases.disease_index",
    # Demographics
    "demographic.age_at_index",
    "demographic.gender",
    "demographic.ethnicity",
    "demographic.race",
    "demographic.ethnicity_race",
    "demographic.vital_status",
    "demographic.days_to_death",
    # Diagnosis
    "diagnoses.age_at_diagnosis",
    "diagnoses.primary_diagnosis",
    "diagnoses.morphology",
    "diagnoses.tumor_stage",
    "diagnoses.tumor_grade",
    "diagnoses.residual_disease",
    "diagnoses.days_to_last_follow_up",
    "diagnoses.progression_or_recurrence",
    # Exposures (far-but-related clinical context)
    "exposures.alcohol_history",
    "exposures.pack_years_smoked",
    "exposures.cigarettes_per_day",
    "exposures.tobacco_smoking_status",
    # Pathology details
    "pathology_details.margin_status",
    "pathology_details.lymph_nodes_positive",
    "pathology_details.lymph_nodes_tested",
    "pathology_details.lymphatic_invasion_present",
    "pathology_details.perineural_invasion_present",
    "pathology_details.vascular_invasion_present",
]

# Keep only those that still exist after cleaning
curated_cols_existing = [c for c in curated_cols if c in df_clean.columns]

df_curated = df_clean[curated_cols_existing].copy()
print(f"\nCurated dataset shape: {df_curated.shape}")
print("Curated columns included:")
for c in curated_cols_existing:
    print(" -", c)

# -----------------------------
# 7. SAVE OUTPUTS
# -----------------------------
df_clean.to_csv(OUT_FULL_CLEAN, index=False)
df_curated.to_csv(OUT_CURATED, index=False)

print(f"\nSaved cleaned full dataset   -> {OUT_FULL_CLEAN}")
print(f"Saved curated smaller dataset -> {OUT_CURATED}")
print("\nDone.")


Loading data...
merged_TCGA_Cohorts shape: (1779, 95)
pathology_detail shape:   (175, 12)

After merge shape: (3651, 106)

Computing column statistics...

Columns to drop (total 12):
['demographic.age_is_obfuscated', 'diagnoses.ajcc_clinical_m', 'diagnoses.ajcc_pathologic_m', 'follow_ups.evidence_of_progression_type', 'follow_ups.evidence_of_recurrence_type', 'follow_ups.progression_or_recurrence', 'molecular_tests.biospecimen_type', 'molecular_tests.laboratory_test', 'molecular_tests.variant_type', 'pathology_details.consistent_pathology_review', 'treatments.chemo_concurrent_to_radiation', 'treatments.clinical_trial_indicator']

Shape before clean: 3651 rows x 105 cols
Shape after  clean: 3651 rows x 93 cols

Drop report saved to: merged_clinical_pathology_drop_report.csv

Curated dataset shape: (3651, 27)
Curated columns included:
 - cases.case_id
 - cases.submitter_id
 - cases.primary_site
 - cases.disease_type
 - cases.index_date
 - cases.disease_index
 - demographic.age_at_index
 

## mutations.csv operations

In [10]:
import pandas as pd

# Load dataset
df = pd.read_csv("mutations.csv", low_memory=False)

# 1. Calculate null ratio for each column
null_ratio = df.isna().mean()

# 2. Identify columns with >90% null values
cols_to_drop = null_ratio[null_ratio > 0.90].index.tolist()

print(f"Columns with >90% null values: {len(cols_to_drop)}")
print("Dropped columns:\n", cols_to_drop)

# 3. Drop these columns
clean_df = df.drop(columns=cols_to_drop)

print(f"Original shape: {df.shape}")
print(f"Cleaned shape:  {clean_df.shape}")

# 4. Save cleaned version
clean_df.to_csv("mutations_cleaned.csv", index=False)

print("\nSaved cleaned dataset as mutations_cleaned.csv!")


Columns with >90% null values: 105
Dropped columns:
 ['dbSNP_Val_Status', 'UniProt_Site', 'UniProt_Natural_Variations', 'UniProt_Experimental_Info', 'COSMIC_overlapping_mutations', 'COSMIC_fusion_genes', 'DrugBank', 'CCLE_ONCOMAP_overlapping_mutations', 'CCLE_ONCOMAP_total_mutations_in_gene', 'CGC_Mutation_Type', 'CGC_Translocation_Partner', 'CGC_Tumor_Types_Somatic', 'CGC_Tumor_Types_Germline', 'CGC_Other_Diseases', 'DNARepairGenes_Role', 'FamilialCancerDatabase_Syndromes', 'MUTSIG_Published_Results', 'OREGANNO_ID', 'OREGANNO_Values', 'i_1000gp3_AA', 'i_1000gp3_AC', 'i_1000gp3_AF', 'i_1000gp3_AFR_AF', 'i_1000gp3_AMR_AF', 'i_1000gp3_AN', 'i_1000gp3_CIEND', 'i_1000gp3_CIPOS', 'i_1000gp3_DP', 'i_1000gp3_EAS_AF', 'i_1000gp3_EUR_AF', 'i_1000gp3_IMPRECISE', 'i_1000gp3_MEINFO', 'i_1000gp3_NS', 'i_1000gp3_SAS_AF', 'i_ACHILLES_Lineage_Results_Top_Genes', 'i_CCLE_ONCOMAP_overlapping_mutations', 'i_CCLE_ONCOMAP_total_mutations_in_gene', 'i_CGC_Cancer Germline Mut', 'i_CGC_Cancer Molecular Geneti

In [11]:
import pandas as pd

# Load dataset
df = pd.read_csv("mutations_cleaned.csv", low_memory=False)

# -----------------------------------------------
# 1. Define unnecessary columns to remove manually
# -----------------------------------------------
drop_cols_manual = [
    "Center", "NCBI_Build", "Strand", "Sequencer",
    "Match_Norm_Seq_Allele1", "Match_Norm_Seq_Allele2",
    "Match_Norm_Validation_Allele1", "Match_Norm_Validation_Allele2",
    "Match_Norm_Validation_Method", "Match_Norm_Validation_Status",
    "Match_Norm_Sample_Barcode",

    # Full-depth counts that duplicate alt/ref counts
    "t_depth_full", "t_alt_count_full", "t_ref_count_full",
    "n_depth_full", "n_alt_count_full", "n_ref_count_full",

    # Overlapping annotations rarely used
    "COSMIC_overlapping_mutations", "COSMIC_fusion_genes",
    "ExAC_AF", "ExAC_FILTER", "ExAC_qt",

    # Predictors with sparse values
    "SIFT", "PolyPhen", "PROVEAN", "CLIN_SIG"
]

# Keep only columns that exist in the df
drop_cols_manual = [c for c in drop_cols_manual if c in df.columns]

print("\nDropping manually identified columns:", len(drop_cols_manual))
print(drop_cols_manual)

df = df.drop(columns=drop_cols_manual)

# ------------------------------------------------
# 2. Automatic cleanup: Drop columns >50% null
# ------------------------------------------------
null_ratio = df.isna().mean()
auto_drop = null_ratio[null_ratio > 0.5].index.tolist()

print("\nDropping columns with >50% null:", len(auto_drop))
print(auto_drop)

df = df.drop(columns=auto_drop)

# ------------------------------------------------
# 3. Save clean dataset
# ------------------------------------------------
df.to_csv("mutations_final_clean.csv", index=False)

print("\nDone! Saved as mutations_final_clean.csv")
print("Final shape:", df.shape)



Dropping manually identified columns: 4
['Center', 'NCBI_Build', 'Strand', 'Sequencer']

Dropping columns with >50% null: 15
['dbSNP_RS', 'UniProt_Region', 'Tumorscape_Amplification_Peaks', 'Tumorscape_Deletion_Peaks', 'TCGAscape_Amplification_Peaks', 'TCGAscape_Deletion_Peaks', 'i_HGNC_Date Name Changed', 'i_HGNC_Date Symbol Changed', 'i_HGNC_Name Synonyms', 'i_HGNC_Previous Names', 'i_HGNC_Previous Symbols', 'i_TCGAscape_Amplification_Peaks', 'i_TCGAscape_Deletion_Peaks', 'i_Tumorscape_Amplification_Peaks', 'i_Tumorscape_Deletion_Peaks']

Done! Saved as mutations_final_clean.csv
Final shape: (13180, 201)


In [12]:
import pandas as pd
import numpy as np

# ---------- 1. Load original file ----------
input_path = "mutations.csv"        # change this to your local path if needed
df = pd.read_csv(input_path, low_memory=False)

print("Original shape:", df.shape)

# ---------- 2. Drop columns with >90% null ----------
null_ratio = df.isna().mean()
cols_drop_null90 = null_ratio[null_ratio > 0.90].index.tolist()

print(f"\nDropping columns with >90% null values: {len(cols_drop_null90)}")
df = df.drop(columns=cols_drop_null90)
print("Shape after >90% null drop:", df.shape)

# ---------- 3. Keep only FUNCTIONAL (non-synonymous) mutations ----------

# Variant_Classification values to KEEP
functional_classes = [
    "Missense_Mutation",
    "Nonsense_Mutation",
    "Frame_Shift_Del",
    "Frame_Shift_Ins",
    "In_Frame_Del",
    "In_Frame_Ins",
    "Splice_Site",
    "Translation_Start_Site",
    "Nonstop_Mutation"
]

if "Variant_Classification" in df.columns:
    before_rows = df.shape[0]
    df = df[df["Variant_Classification"].isin(functional_classes)]
    after_rows = df.shape[0]
    print(f"\nFiltered functional Variant_Classification:")
    print(f"  Rows before: {before_rows}")
    print(f"  Rows after:  {after_rows}")
else:
    print("\n⚠ 'Variant_Classification' not found - skipping functional filter.")

# ---------- 4. Compute Variant Allele Frequency (VAF) ----------

if {"t_alt_count", "t_ref_count"}.issubset(df.columns):
    depth = df["t_alt_count"] + df["t_ref_count"]
    df["VAF"] = np.where(depth > 0, df["t_alt_count"] / depth, np.nan)
    print("\nComputed VAF = t_alt_count / (t_alt_count + t_ref_count)")
else:
    print("\n⚠ t_alt_count and/or t_ref_count columns not found - skipping VAF.")

# ---------- 5. Keep only essential columns ----------

essential_cols = [
    "Hugo_Symbol",
    "Entrez_Gene_Id",
    "Chromosome",
    "Start_position",
    "End_position",
    "Reference_Allele",
    "Tumor_Seq_Allele1",
    "Tumor_Seq_Allele2",
    "Variant_Classification",
    "Variant_Type",
    "Tumor_Sample_Barcode",
    "HGVSc",
    "HGVSp_Short",
    "t_alt_count",
    "t_ref_count",
    "n_alt_count",
    "n_ref_count",
    "VAF"
]

existing_cols = [c for c in essential_cols if c in df.columns]
missing_cols = [c for c in essential_cols if c not in df.columns]

print(f"\nEssential columns requested: {len(essential_cols)}")
print(f"Columns found and kept:       {len(existing_cols)}")
print("Kept columns:", existing_cols)
if missing_cols:
    print("Missing (not in file):", missing_cols)

df_reduced = df[existing_cols].copy()

print("\nFinal reduced shape:", df_reduced.shape)

# ---------- 6. Save reduced dataset ----------
output_path = "mutations_reduced.csv"   # change if you want another name/path
df_reduced.to_csv(output_path, index=False)
print(f"\nSaved reduced dataset to: {output_path}")


Original shape: (13180, 325)

Dropping columns with >90% null values: 105
Shape after >90% null drop: (13180, 220)

Filtered functional Variant_Classification:
  Rows before: 13180
  Rows after:  9637

Computed VAF = t_alt_count / (t_alt_count + t_ref_count)

Essential columns requested: 18
Columns found and kept:       14
Kept columns: ['Hugo_Symbol', 'Entrez_Gene_Id', 'Chromosome', 'Start_position', 'End_position', 'Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'Variant_Classification', 'Variant_Type', 'Tumor_Sample_Barcode', 't_alt_count', 't_ref_count', 'VAF']
Missing (not in file): ['HGVSc', 'HGVSp_Short', 'n_alt_count', 'n_ref_count']

Final reduced shape: (9637, 14)

Saved reduced dataset to: mutations_reduced.csv


In [16]:
import pandas as pd
import numpy as np

# ======================================================
# 0. FILE PATHS (EDIT IF NEEDED)
# ======================================================

mut_path = "mutations_reduced.csv"
pathology_path = "Pathology_Cohorts.csv"
epigenetic_path = "combined_epigenetic_score_matrix.csv"

# ======================================================
# 1. LOAD DATASETS
# ======================================================

mut = pd.read_csv(mut_path, low_memory=False)
path = pd.read_csv(pathology_path, low_memory=False)
epi = pd.read_csv(epigenetic_path, low_memory=False)

print("Loaded:")
print("  mutations:   ", mut.shape)
print("  pathology:   ", path.shape)
print("  epigenetics: ", epi.shape)

# Quick sanity check for key columns
print("\n[INFO] mutation columns:", mut.columns.tolist())
print("[INFO] pathology columns:", path.columns.tolist()[:10], "...")  # first 10
print("[INFO] epigenetic columns:", epi.columns.tolist()[:10], "...")


# ======================================================
# 2. STANDARDISE BARCODES
# ======================================================
# TCGA barcode levels:
# - Case (patient):   TCGA-BA-4074         (12 chars)
# - Sample:           TCGA-BA-4074-01      (15 chars)
# - Full aliquot:     TCGA-BA-4074-01A-... (long)

# -------- 2.1 From mutation data --------
if "Tumor_Sample_Barcode" not in mut.columns:
    raise KeyError("Tumor_Sample_Barcode not found in mutations_reduced.csv")

# Sample-level ID for epigenetic merge (first 15 chars)
mut["Sample_ID_15"] = mut["Tumor_Sample_Barcode"].astype(str).str[:15]

# Case-level ID for pathology merge (first 12 chars)
mut["Case_ID_12"] = mut["Tumor_Sample_Barcode"].astype(str).str[:12]


# -------- 2.2 From pathology data --------
if "cases.submitter_id" not in path.columns:
    raise KeyError("cases.submitter_id not found in Pathology_Cohorts.csv")

path["Case_ID_12"] = path["cases.submitter_id"].astype(str).str[:12]


# -------- 2.3 From epigenetic data --------
# Assume: first column = gene name, remaining columns = TCGA sample barcodes.
epi_gene_col = epi.columns[0]
epi = epi.rename(columns={epi_gene_col: "Gene"})
epi.set_index("Gene", inplace=True)

# Epigenetic columns should already look like TCGA-XX-XXXX-01
# If any have longer suffixes, truncate to first 15 chars
epi_cols_renamed = {}
for col in epi.columns:
    if col.startswith("TCGA"):
        epi_cols_renamed[col] = col[:15]
epi.rename(columns=epi_cols_renamed, inplace=True)

print("\n[INFO] Example mutation Sample_ID_15:", mut["Sample_ID_15"].head().tolist())
print("[INFO] Example mutation Case_ID_12:", mut["Case_ID_12"].head().tolist())
print("[INFO] Example epi columns:", epi.columns[:10].tolist())


# ======================================================
# 3. FILE 1 — MERGE MUTATIONS WITH PATHOLOGY
# ======================================================

print("\n=== Building mutations_plus_pathology.csv ===")

# We'll keep all mutation columns + all pathology columns
mut_plus_path = mut.merge(
    path,
    on="Case_ID_12",
    how="inner",  # only patients present in both
    suffixes=("_mut", "_path")
)

print("Merged mutations + pathology shape:", mut_plus_path.shape)

mut_plus_path.to_csv("mutations_plus_pathology.csv", index=False)
print("Saved: mutations_plus_pathology.csv")


# ======================================================
# 4. FILE 2 — BUILD MUTATION MATRIX + MERGE WITH EPIGENETIC MATRIX
# ======================================================

print("\n=== Building mutations_plus_epigenetics.csv ===")

# 4.1 Build a gene × sample mutation matrix (binary 0/1)
# One row per gene, one column per sample (Sample_ID_15)
if not {"Hugo_Symbol", "Sample_ID_15"}.issubset(mut.columns):
    raise KeyError("Hugo_Symbol and/or Sample_ID_15 missing in mutation data.")

mutation_matrix = (
    mut
    .groupby(["Hugo_Symbol", "Sample_ID_15"])
    .size()
    .unstack(fill_value=0)
)

print("Initial mutation matrix shape (genes x samples):", mutation_matrix.shape)

# Optional: convert counts >0 to 1 (pure binary mutated/not mutated)
mutation_matrix = (mutation_matrix > 0).astype(int)


# 4.2 Align genes between mutation matrix and epigenetic matrix
common_genes = mutation_matrix.index.intersection(epi.index)
print("Number of common genes between mutation & epigenetic:", len(common_genes))

mut_mat_common = mutation_matrix.loc[common_genes]
epi_common = epi.loc[common_genes]

print("mut_mat_common shape:", mut_mat_common.shape)
print("epi_common shape:", epi_common.shape)

# 4.3 Concatenate along columns: for each gene,
# [mutation features (samples)] + [epigenetic features (samples)]
mut_plus_epi = pd.concat([mut_mat_common, epi_common], axis=1)

print("Final mutations_plus_epigenetics shape:", mut_plus_epi.shape)

mut_plus_epi.to_csv("mutations_plus_epigenetics.csv")
print("Saved: mutations_plus_epigenetics.csv")


Loaded:
  mutations:    (9637, 14)
  pathology:    (3651, 93)
  epigenetics:  (19199, 89)

[INFO] mutation columns: ['Hugo_Symbol', 'Entrez_Gene_Id', 'Chromosome', 'Start_position', 'End_position', 'Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'Variant_Classification', 'Variant_Type', 'Tumor_Sample_Barcode', 't_alt_count', 't_ref_count', 'VAF']
[INFO] pathology columns: ['cases.case_id', 'cases.consent_type', 'cases.days_to_consent', 'cases.disease_type', 'cases.index_date', 'cases.lost_to_followup', 'cases.primary_site', 'demographic.age_at_index', 'demographic.cause_of_death', 'demographic.country_of_residence_at_enrollment'] ...
[INFO] epigenetic columns: ['gene_id', 'TCGA-BA-4074-01', 'TCGA-BA-4077-01', 'TCGA-BA-5153-01', 'TCGA-BA-5556-01', 'TCGA-BA-5557-01', 'TCGA-BA-6872-01', 'TCGA-BA-6873-01', 'TCGA-BA-7269-01', 'TCGA-BB-4224-01'] ...

[INFO] Example mutation Sample_ID_15: ['TCGA-BA-4074-01', 'TCGA-BA-4074-01', 'TCGA-BA-4074-01', 'TCGA-BA-4074-01', 'TCGA-BA-4074-

In [21]:
import pandas as pd

# Load your multi-omics file
df = pd.read_csv("mutations_plus_epigenetics.csv", index_col=0)

# ---------------------------------------------------------
# 1. Identify mutation and epigenetic columns
# ---------------------------------------------------------

mut_cols = df.columns[:82]     # first 82 = mutation block
epi_cols = df.columns[82:]     # last 88 = epigenetic block

df_mut = df[mut_cols].copy()
df_epi = df[epi_cols].copy()

# ---------------------------------------------------------
# 2. Clean epigenetic column names (remove .1)
# ---------------------------------------------------------

clean_epi_cols = []

for c in df_epi.columns:
    if c.endswith(".1"):
        clean_epi_cols.append(c[:-2])   # remove ".1"
    else:
        clean_epi_cols.append(c)

df_epi.columns = clean_epi_cols

# ---------------------------------------------------------
# 3. Remove -11 samples (normal tissue, not tumour)
# ---------------------------------------------------------

df_epi = df_epi[[c for c in df_epi.columns if not c.endswith("-11")]]

# ---------------------------------------------------------
# 4. Find intersection
# ---------------------------------------------------------

mutation_samples = set(df_mut.columns)
epigenetic_samples = set(df_epi.columns)

common_samples = sorted(list(mutation_samples & epigenetic_samples))

print("\nIntersection sample count:", len(common_samples))
print("Common samples:", common_samples)

# ---------------------------------------------------------
# 5. Multiply mutation × epigenetic values
# ---------------------------------------------------------

interaction_df = df_mut[common_samples] * df_epi[common_samples]

interaction_df.to_csv("mutation_epigenetic_interaction_matrix.csv")

print("\nSaved: mutation_epigenetic_interaction_matrix.csv")
print("Final shape:", interaction_df.shape)



Intersection sample count: 82
Common samples: ['TCGA-BA-4074-01', 'TCGA-BA-4077-01', 'TCGA-BA-5153-01', 'TCGA-BA-5556-01', 'TCGA-BA-5557-01', 'TCGA-BA-6872-01', 'TCGA-BA-6873-01', 'TCGA-BA-7269-01', 'TCGA-BB-4224-01', 'TCGA-BB-4225-01', 'TCGA-BB-4228-01', 'TCGA-CN-4726-01', 'TCGA-CN-4729-01', 'TCGA-CN-4737-01', 'TCGA-CN-4740-01', 'TCGA-CN-4741-01', 'TCGA-CN-4742-01', 'TCGA-CN-5359-01', 'TCGA-CN-5364-01', 'TCGA-CN-6016-01', 'TCGA-CN-6018-01', 'TCGA-CN-6995-01', 'TCGA-CQ-5330-01', 'TCGA-CQ-5331-01', 'TCGA-CQ-5332-01', 'TCGA-CQ-5334-01', 'TCGA-CQ-7065-01', 'TCGA-CQ-7068-01', 'TCGA-CR-6477-01', 'TCGA-CR-6478-01', 'TCGA-CR-6481-01', 'TCGA-CR-6487-01', 'TCGA-CR-6488-01', 'TCGA-CR-6493-01', 'TCGA-CR-7367-01', 'TCGA-CR-7368-01', 'TCGA-CR-7372-01', 'TCGA-CR-7373-01', 'TCGA-CR-7376-01', 'TCGA-CR-7379-01', 'TCGA-CR-7383-01', 'TCGA-CR-7385-01', 'TCGA-CR-7391-01', 'TCGA-CR-7392-01', 'TCGA-CR-7394-01', 'TCGA-CR-7401-01', 'TCGA-CV-5442-01', 'TCGA-CV-5971-01', 'TCGA-CV-5973-01', 'TCGA-CV-5979-01', 'T

## Add tag to Pathology_Cohort.csv

In [2]:
import pandas as pd

# Load files
clinical = pd.read_csv("clinical.csv")
patho = pd.read_csv("Pathology_Cohorts.csv")

# Extract mapping
clinical_map = clinical[["cases.case_id", "cases.submitter_id"]].drop_duplicates()

# Merge
updated = patho.merge(clinical_map, on="cases.case_id", how="left", suffixes=("", "_from_clinical"))

# Fill or create submitter_id
if "cases.submitter_id" in updated.columns:
    updated["cases.submitter_id"] = updated["cases.submitter_id"].fillna(updated["cases.submitter_id_from_clinical"])
else:
    updated.rename(columns={"cases.submitter_id_from_clinical": "cases.submitter_id"}, inplace=True)

# Drop helper column
if "cases.submitter_id_from_clinical" in updated.columns:
    updated.drop(columns=["cases.submitter_id_from_clinical"], inplace=True)

# Make cases.submitter_id the first column
cols = ["cases.submitter_id"] + [c for c in updated.columns if c != "cases.submitter_id"]
updated = updated[cols]

# Save updated file
output = "Pathology_Cohorts_UPDATED_firstcol.csv"
updated.to_csv(output, index=False)

output


'Pathology_Cohorts_UPDATED_firstcol.csv'